# Week 7 Tutorial: Spatial Mapping & Visualization

## 0. Install and environment notes

Recommended environment (once):

```bash
conda activate geopy
conda install -y -c conda-forge xarray netcdf4 matplotlib cartopy
```

If cartopy is hard to install, this notebook still works with `pcolormesh` and `contourf` plotted on plain lon/lat axes (no map features).

## 1. Imports and quick checks

We import the typical libraries and check for `cartopy`. If `cartopy` is missing, we print a friendly message and continue without map coastlines.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt

_have_cartopy = True
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
except Exception:
    _have_cartopy = False
    print('cartopy not available: map coastlines/features will be skipped. To enable, run: conda install -c conda-forge cartopy')

## 2. Create or load a test spatial netCDF

We create a small gridded netCDF dataset (time × lat × lon) with a clear spatial gradient plus random noise. When you replace this with your real climate netCDF file, use `xr.open_dataset('yourfile.nc')` instead.

In [ ]:
from datetime import datetime, timedelta
import os
fname = 'spatial_practice.nc'
if os.path.exists(fname):
    os.remove(fname)

nlat, nlon = 40, 60
lats = np.linspace(-10, 60, nlat)
lons = np.linspace(-140, -40, nlon)
ntime = 12
base_date = datetime(2024,1,1)
times = [base_date + timedelta(days=30*i) for i in range(ntime)]
lon_grid, lat_grid = np.meshgrid(lons, lats)
field = np.zeros((ntime, nlat, nlon))
for t in range(ntime):
    seasonal = 5.0 * np.sin(2*np.pi*(t/12.0))
    lat_grad = (lat_grid - 25.0) * 0.1
    hotspot = 8.0 * np.exp(-((lat_grid-30)**2 + (lon_grid+90)**2)/200.0)
    noise = np.random.normal(0, 0.8, size=(nlat, nlon))
    field[t,:,:] = 15 + seasonal + lat_grad + hotspot + noise

import netCDF4 as nc
ds = nc.Dataset(fname, 'w')
ds.createDimension('time', ntime)
ds.createDimension('lat', nlat)
ds.createDimension('lon', nlon)

t_var = ds.createVariable('time','i4',('time',))
lat_var = ds.createVariable('lat','f4',('lat',))
lon_var = ds.createVariable('lon','f4',('lon',))
val_var = ds.createVariable('temperature','f4',('time','lat','lon'))

units = 'days since 1970-01-01'
t_var.units = units
t_var[:] = nc.date2num(times, units)
lat_var[:] = lats
lon_var[:] = lons
val_var.units = 'C'
val_var.long_name = 'Synthetic monthly temperature'
val_var[:] = field

ds.close()
print('Wrote', fname)


## 3. Open with xarray and inspect

Read the file and inspect dimensions and coordinates. Always check `ds.coords` and `ds.dims` before plotting.

In [ ]:
ds = xr.open_dataset('spatial_practice.nc')
# display dataset
print(ds)

# convert times to datetime objects for indexing
import netCDF4 as _nc
import pandas as pd

## 4. Quick helper: select a time slice, lat/lon arrays, and the 2D array

We will pick one time index for spatial plotting. Replace `time_index` with the index you want (e.g., month 0..11).

In [ ]:
time_index = 5
field2d = ds['temperature'].isel(time=time_index)
lat = ds['lat'].values
lon = ds['lon'].values
print('Field shape:', field2d.shape)
print('Lat shape, Lon shape:', lat.shape, lon.shape)


## 5. Basic pcolormesh plot (fast, flexible)

`pcolormesh` draws colored quadrilaterals for each grid cell. It's often the most appropriate for irregular grids. We'll set a sensible colorbar range using percentiles to avoid outlier-driven scaling.

In [ ]:
Lon, Lat = np.meshgrid(lon, lat)
vmin = np.nanpercentile(field2d, 2)
vmax = np.nanpercentile(field2d, 98)

plt.figure(figsize=(8,4))
if _have_cartopy:
    ax = plt.axes(projection=ccrs.PlateCarree())
    pcm = ax.pcolormesh(Lon, Lat, field2d, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
else:
    ax = plt.gca()
    pcm = ax.pcolormesh(Lon, Lat, field2d, vmin=vmin, vmax=vmax)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

cbar = plt.colorbar(pcm, orientation='vertical', pad=0.02)
cbar.set_label(f"Temperature ({ds['temperature'].units})")
plt.title(f"Temperature (time index {time_index}) - pcolormesh")
plt.tight_layout()
plt.show()


## 6. Contourf plot (smooth isolines)

`contourf` draws filled contours and is helpful to highlight gradients and isolines. Choose contour levels deliberately.

In [ ]:
levels = np.linspace(vmin, vmax, 12)

plt.figure(figsize=(8,4))
if _have_cartopy:
    ax = plt.axes(projection=ccrs.PlateCarree())
    cf = ax.contourf(Lon, Lat, field2d, levels=levels, transform=ccrs.PlateCarree(), extend='both')
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
else:
    ax = plt.gca()
    cf = ax.contourf(Lon, Lat, field2d, levels=levels, extend='both')
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')

cbar = plt.colorbar(cf, orientation='vertical', pad=0.02)
cbar.set_label('Temperature (C)')
plt.title('Temperature - contourf')
plt.tight_layout()
plt.show()


## 7. Contour lines over pcolormesh (best of both)

Overlay contour lines on a pcolormesh to get both area shading and clear isolines.

In [ ]:
plt.figure(figsize=(8,4))
if _have_cartopy:
    ax = plt.axes(projection=ccrs.PlateCarree())
    pcm = ax.pcolormesh(Lon, Lat, field2d, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
    cs = ax.contour(Lon, Lat, field2d, levels=levels, colors='k', linewidths=0.6, transform=ccrs.PlateCarree())
    ax.clabel(cs, fmt='%.1f', inline=True, fontsize=8)
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
else:
    ax = plt.gca()
    pcm = ax.pcolormesh(Lon, Lat, field2d, vmin=vmin, vmax=vmax)
    cs = ax.contour(Lon, Lat, field2d, levels=levels, colors='k', linewidths=0.6)
    ax.clabel(cs, fmt='%.1f', inline=True, fontsize=8)
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')

plt.colorbar(pcm, pad=0.02).set_label('Temperature (C)')
plt.title('Temperature: pcolormesh + contour lines')
plt.tight_layout()
plt.show()


## 8. Overlay scatter points (stations) on the map

We simulate a few station locations (lat/lon pairs) and overlay them on the spatial map to show how to combine gridded fields with point observations.

In [ ]:
# create some station locations (random-ish but within the grid)
nstation = 12
np.random.seed(1)
station_lats = np.random.uniform(lat.min(), lat.max(), nstation)
station_lons = np.random.uniform(lon.min(), lon.max(), nstation)

# sample the field value at the nearest grid point for each station
station_vals = []
for la, lo in zip(station_lats, station_lons):
    i = np.abs(lat - la).argmin()
    j = np.abs(lon - lo).argmin()
    station_vals.append(field2d.values[i, j])

plt.figure(figsize=(8,4))
if _have_cartopy:
    ax = plt.axes(projection=ccrs.PlateCarree())
    pcm = ax.pcolormesh(Lon, Lat, field2d, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
    ax.scatter(station_lons, station_lats, c=station_vals, cmap='viridis', edgecolor='k', transform=ccrs.PlateCarree())
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
else:
    ax = plt.gca()
    pcm = ax.pcolormesh(Lon, Lat, field2d, vmin=vmin, vmax=vmax)
    ax.scatter(station_lons, station_lats, c=station_vals, cmap='viridis', edgecolor='k')
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')

plt.colorbar(pcm, pad=0.02).set_label('Temperature (C)')
plt.title('Temperature with station overlay')
plt.tight_layout()
plt.show()


## 9. Masks and handling missing data

Real datasets often have missing values (land/ocean masks, coastlines). We'll demonstrate masking a region and how to ensure maps display correctly by using `np.ma.masked_where`. 

In [ ]:
mask = np.ma.masked_where(Lat > 50, field2d)

plt.figure(figsize=(8,4))
if _have_cartopy:
    ax = plt.axes(projection=ccrs.PlateCarree())
    pcm = ax.pcolormesh(Lon, Lat, mask, transform=ccrs.PlateCarree())
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
else:
    ax = plt.gca()
    pcm = ax.pcolormesh(Lon, Lat, mask)
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')

plt.colorbar(pcm, pad=0.02).set_label('Temperature (masked)')
plt.title('Masked temperature (Lat > 50 masked)')
plt.tight_layout()
plt.show()


<font size=5> If you have no problem in spatial plotting, next step is to explore the data I provided below.

    

<font size=4> The dataset is on cloud (https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc), below is how to open and read the data without downloading it.

In [ ]:
url = "https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
ds = xr.open_dataset(url)
print(ds)